# Preprocessing

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, MaxPool2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Model
from keras import regularizers
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow._api.v2.compat.v1 as tf
tf.disable_v2_behavior()
import os
import cv2
import PIL
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/')

MessageError: ignored

In [ ]:
#Importing Datasets
img_dir='brain_tumor/'
no_images=os.listdir(img_dir + 'no/')
yes_images=os.listdir(img_dir + 'yes/')
dataset=[]
lab=[]

#No tumour images
for image_name in no_images:
    image=cv2.imread(img_dir + 'no/' +image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((112,112))
    dataset.append(np.array(image))
    lab.append(0)

#Tumour images
for image_name in yes_images:
    image=cv2.imread(img_dir + 'yes/' +image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((112,112))
    dataset.append(np.array(image))
    lab.append(1)

data=np.asarray(dataset)
l=np.asarray(lab)
print(data.shape, l.shape)
print('Total Number of Image: ',len(l))



In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data,l, test_size=0.3, shuffle=True, random_state=0, stratify=l)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255

# Reshape 

img_width  = x_train.shape[1]
img_height = x_train.shape[2]
num_channels = x_train.shape[3]
# x_train = x_train.reshape(x_train.shape[0], img_height, img_width, num_channels)
# x_test = x_test.reshape(x_test.shape[0], img_height, img_width, num_channels)
input_shape = (img_height, img_width, num_channels)

# Grid Search

In [ ]:
def classifier():
  model = Sequential()
  #
  model.add(Conv2D(32, (5,5), activation ='relu', input_shape = (112,112,3)))
  model.add(MaxPool2D(pool_size=(2,2)))
  #
  model.add(Conv2D(64, (3,3),activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2), ))

  # fully connected
  model.add(Flatten())
  model.add(Dense(256, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation = "softmax"))
  model.compile(optimizer = 'adam' , loss = "sparse_categorical_crossentropy", metrics=["accuracy"])
  return model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience = 5)

In [ ]:
from sklearn.model_selection import GridSearchCV 
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn = classifier, epochs=15)
#What hyperparameter we want to play with
parameters = {'batch_size': [5, 10, 15, 20]}
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           cv = 3)
grid_search = grid_search.fit(x_train, y_train, validation_split = 0.2, callbacks=[early_stopping])

In [ ]:
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Training

In [ ]:
model = Sequential()
#
model.add(Conv2D(32, (5,5), activation ='relu', input_shape = (112,112,3)))
#model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3),activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2), ))

# fully connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))
model.compile(optimizer = 'adam' , loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience = 4)

In [ ]:
model.compile(optimizer = 'adam' , loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

history= model.fit(x_train, y_train, epochs = 20, batch_size = 15, validation_split = 0.2, callbacks=[early_stopping])


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.title('model acc')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(np.round(y_pred),axis=1)


In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
correct = np.where(y_pred==y_test)[0]
print( "Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[correct], cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(y_pred[correct], y_test[correct]))
plt.tight_layout()

In [ ]:
incorrect = np.where(y_pred!=y_test)[0]
print( "Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[incorrect], cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(y_pred[incorrect], y_test[incorrect]))
plt.tight_layout()

# Generated Pre Processing

In [ ]:
#Importing Datasets
img_dir='brain_tumor_extra/brain_tumor/'
no_images=os.listdir(img_dir + 'no/')
yes_images=os.listdir(img_dir + 'yes/')
dataset=[]
lab=[]

#No tumour images
for image_name in no_images:
    image=cv2.imread(img_dir + 'no/' +image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((112,112))
    dataset.append(np.array(image))
    lab.append(0)

#Tumour images
for image_name in yes_images:
    image=cv2.imread(img_dir + 'yes/' +image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((112,112))
    dataset.append(np.array(image))
    lab.append(1)

data=np.asarray(dataset)
l=np.asarray(lab)
print(data.shape, l.shape)
print('Total Number of Image: ',len(l))



In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data,l, test_size=0.3, shuffle=True, random_state=0, stratify=l)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255

# Reshape 

img_width  = x_train.shape[1]
img_height = x_train.shape[2]
num_channels = x_train.shape[3]
# x_train = x_train.reshape(x_train.shape[0], img_height, img_width, num_channels)
# x_test = x_test.reshape(x_test.shape[0], img_height, img_width, num_channels)
input_shape = (img_height, img_width, num_channels)

# Generated Training

In [ ]:
model = Sequential()
#
model.add(Conv2D(32, (5,5), activation ='relu', input_shape = (112,112,3)))
#model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3),activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2), ))

# fully connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))
model.compile(optimizer = 'adam' , loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience = 4)

In [ ]:
model.compile(optimizer = 'adam' , loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

history= model.fit(x_train, y_train, epochs = 20, batch_size = 15, validation_split = 0.2, callbacks=[early_stopping])


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.title('model acc')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(np.round(y_pred),axis=1)


In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
correct = np.where(y_pred==y_test)[0]
print( "Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[correct], cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(y_pred[correct], y_test[correct]))
plt.tight_layout()

In [ ]:
incorrect = np.where(y_pred!=y_test)[0]
print( "Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[incorrect], cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(y_pred[incorrect], y_test[incorrect]))
plt.tight_layout()

In [ ]:
print(model.summary())